<a href="https://colab.research.google.com/github/mimendez00/mimendez_public_data/blob/main/Langchain_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [200]:
import os
os.environ['OPENAI_API_KEY'] = ""
os.environ['COHERE_API_KEY'] = ""

In [40]:
# All of this was taken from https://langchain.readthedocs.io/en/latest/modules/document_loaders/examples/unstructured_file.html

## Install package
!pip install "unstructured[local-inference]"
!pip install "detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2"
!pip install layoutparser[layoutmodels,tesseract]
!pip install langchain
!pip install cohere
!pip install pysrt
!pip install openai
!pip install chromadb
import nltk
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git (to revision v0.6) to /tmp/pip-install-akqu_m77/detectron2_40e1deec26f949a4a3877ddba634c2a9
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-install-akqu_m77/detectron2_40e1deec26f949a4a3877ddba634c2a9
  Running command git checkout -q d1e04565d3bec8719335b88be9e9b961bf3ec464
  Resolved https://github.com/facebookresearch/detectron2.git to commit d1e04565d3bec8719335b88be9e9b961bf3ec464
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, h

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [201]:
!wget https://raw.githubusercontent.com/mimendez00/mimendez_public_data/main/Andrej-v3.txt

--2023-03-08 01:47:04--  https://raw.githubusercontent.com/mimendez00/mimendez_public_data/main/Andrej-v3.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 230158 (225K) [text/plain]
Saving to: ‘Andrej-v3.txt.1’

Andrej-v3.txt.1     100%[===================>] 224.76K  --.-KB/s    in 0.02s   

2023-03-08 01:47:04 (12.6 MB/s) - ‘Andrej-v3.txt.1’ saved [230158/230158]



In [191]:
from langchain.document_loaders import TextLoader

loader = TextLoader("Andrej-v3.txt")
docs = loader.load()

In [192]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 200,
    separator = "\n"
)

texts = text_splitter.split_documents(docs)


In [193]:
print(len(texts))
print(texts[0])

284
page_content='Andrej Karpathy said "think it\'s possible that physics has exploits and we should be trying to find them. arranging some kind of a crazy quantum mechanical system that somehow gives you buffer overflow somehow gives you a rounding error and the floating point. Synthetic intelligences are kind of like the next stage of development. And I don\'t know where it leads to like at some point, I suspect the universe is some kind of a puzzle, these synthetic AIS will uncover that puzzle and solve it."\nLex Fridman said "The following is a conversation with Andrej Karpathy. Previously, the director of AI at Tesla. And before that, at open AI, and Stanford, he is one of the greatest scientists, engineers and educators in the history of artificial intelligence. This is the LEX Friedman podcast to support it, please check out our sponsors. And now, dear friends, here\'s Andrej Karpathy thing. What is a neural network? And why does it seem to do such a surprisingly good job of lea

In [194]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.vectorstores import Chroma
from langchain import OpenAI, VectorDBQA, Cohere
from langchain.chains import VectorDBQAWithSourcesChain
import json

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, metadatas=[{"source": f"{i}-pl"} for i in range(len(texts))])



Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


DEBUG:Chroma:Index not found
DEBUG:Chroma:Index saved to .chroma/index/index.bin
DEBUG:Chroma:Index saved to .chroma/index/index.bin


In [195]:
print(len(texts))
print(texts[0])

284
page_content='Andrej Karpathy said "think it\'s possible that physics has exploits and we should be trying to find them. arranging some kind of a crazy quantum mechanical system that somehow gives you buffer overflow somehow gives you a rounding error and the floating point. Synthetic intelligences are kind of like the next stage of development. And I don\'t know where it leads to like at some point, I suspect the universe is some kind of a puzzle, these synthetic AIS will uncover that puzzle and solve it."\nLex Fridman said "The following is a conversation with Andrej Karpathy. Previously, the director of AI at Tesla. And before that, at open AI, and Stanford, he is one of the greatest scientists, engineers and educators in the history of artificial intelligence. This is the LEX Friedman podcast to support it, please check out our sponsors. And now, dear friends, here\'s Andrej Karpathy thing. What is a neural network? And why does it seem to do such a surprisingly good job of lea

In [196]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
qa_chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="map_rerank", return_intermediate_steps=True)

qa = VectorDBQAWithSourcesChain(combine_documents_chain=qa_chain, vectorstore=docsearch, verbose=True)
#qa = VectorDBQAWithSourcesChain.from_chain_type(OpenAI(temperature=0), chain_type="refine", vectorstore=docsearch, verbose=True)

Exiting: Cleaning up .chroma directory


In [199]:
query_question = "Did Lex mention Optimus?"


docresults = docsearch.similarity_search(query_question)
for i in range(len(docresults)):
  print(docresults[i].page_content)

qa({"question": query_question}, return_only_outputs=True)

DEBUG:Chroma:time to pre process our knn query: 3.5762786865234375e-06
DEBUG:Chroma:time to run knn query: 0.0004305839538574219


Andrej Karpathy said "I guess occasionally. I do enjoy movies that don't feature or like Anchorman that has. That's the thinker, man. It's so good."
Lex Fridman said "I don't understand. Um, he's speaking of AGI because I don't understand why Will Ferrell is so funny. It doesn't make sense. It doesn't compute. There's just something about him. And he's a singular human because you don't get that many comedies these days and I wonder if it has to do about the culture or the like the the machine of Hollywood or does it have to do with just we got lucky with certain people in comedy. It came together because he is a single human. That was a ridiculous tangent. I apologize. But you mentioned humanoid robot. So what do you think about Optimus about About Do you think we'll have robots in the factory in the home in 1020 3040 50 years?"
Lex Fridman said "there's also dance between the genius and the personal, like, what it means to love another human being. And there's a lot of things there i

DEBUG:Chroma:time to pre process our knn query: 2.1457672119140625e-06
DEBUG:Chroma:time to run knn query: 0.0011134147644042969



> Finished chain.


{'answer': ' Yes, Lex mentioned Optimus.', 'sources': ''}